<a href="https://colab.research.google.com/github/vitsiupia/business-meetings-summarization/blob/main/ami_meetings_making_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pobieramy dane z githuba

!wget 'https://github.com/vitsiupia/business-meetings-summarization/raw/main/ami_meetings_untouched.zip'
# pamiętajmy o tym, że linkujemy do pliku raw

--2023-06-10 06:56:25--  https://github.com/vitsiupia/business-meetings-summarization/raw/main/ami_meetings_untouched.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vitsiupia/business-meetings-summarization/main/ami_meetings_untouched.zip [following]
--2023-06-10 06:56:25--  https://raw.githubusercontent.com/vitsiupia/business-meetings-summarization/main/ami_meetings_untouched.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1901391 (1.8M) [application/zip]
Saving to: ‘ami_meetings_untouched.zip’

ami_meetings_untouc 100%[===================>]   1.81M  --.-KB/s    in 0.06s   

2023-06-10

In [2]:
# Rozpakowujemy je
import zipfile
# Unzip the file
zip_ref = zipfile.ZipFile('ami_meetings_untouched.zip')
zip_ref.extractall()
zip_ref.close()

In [3]:
import os
import pandas as pd

# Set the paths to the main folder and the train and val subfolders
main_folder = 'ami_meetings_untouched'
train_folder = os.path.join(main_folder, 'train')
val_folder = os.path.join(main_folder, 'val')

# Initialize lists to store the data
data = []

# Process the train data
for root, dirs, files in os.walk(os.path.join(train_folder, 'transcripts')):
    for file in files:
        if file.endswith('.transcript.txt'):
            meeting_id = file.split('.')[0]
            transcript_path = os.path.join(root, file)
            summary_path = os.path.join(train_folder, 'summaries', f'{meeting_id}.abssumm.txt')

            with open(transcript_path, 'r') as transcript_file:
                transcript = transcript_file.read().strip()

            with open(summary_path, 'r') as summary_file:
                summary = summary_file.read().strip()

            data.append([transcript, summary, meeting_id])

# Process the validation data
for root, dirs, files in os.walk(os.path.join(val_folder, 'transcripts')):
    for file in files:
        if file.endswith('.transcript.txt'):
            meeting_id = file.split('.')[0]
            transcript_path = os.path.join(root, file)
            summary_path = os.path.join(val_folder, 'summaries', f'{meeting_id}.abssumm.txt')

            with open(transcript_path, 'r') as transcript_file:
                transcript = transcript_file.read().strip()

            with open(summary_path, 'r') as summary_file:
                summary = summary_file.read().strip()

            data.append([transcript, summary, meeting_id])

# Create the DataFrame
df = pd.DataFrame(data, columns=['transcript', 'summary', 'meeting_id'])

# Perform train-val split
train_df = df[df['meeting_id'].str.startswith('train/')]
val_df = df[df['meeting_id'].str.startswith('val/')]

# Remove the 'train/' and 'val/' prefixes from the meeting_id column
train_df['meeting_id'] = train_df['meeting_id'].str.replace('train/', '')
val_df['meeting_id'] = val_df['meeting_id'].str.replace('val/', '')

# Reset the index for both DataFrames
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [4]:
df

,transcript,summary,meeting_id
0,Alima. Okay. Um I don't know which one of thes...,The project manager opened the meeting and int...,ES2008a
1,I used to eat it. Yep. We've got a cool protot...,The interface specialist and industrial design...,ES2007d
2,"Did you manage? Okay. Ah. Hello. No, just uh a...","For the conceptual design, the ID suggested to...",TS3012c
3,"That should hopefully do the trick, um. 'Kay. ...",The project manager recapped the decisions mad...,ES2003d
4,I'm sorry. What was the where's the L_E_D_? Oh...,The project manager opens the meeting by stati...,ES2015d
...,...,...,...
137,That's my task. Okay. Uh I've Where have I put...,The project manager opened the meeting by expl...,TS3010b
138,Okay. Hi everybody and welcome to our kick-off...,The Project Manager introduced the project to ...,ES2010a
139,Also lots of mobile phones have got a lot of t...,The project manager opens the meeting by recap...,ES2012c
140,Bonjour. Why? Okay. Check check check check. T...,The Project Manager reviewed the minutes from ...,TS3004c


In [6]:
import pandas as pd
import re

# Assuming 'df' is the DataFrame containing the 'transcript', 'summary', and 'meeting_id' columns

# Lowercase the text in 'transcript' and 'summary' columns
df['transcript'] = df['transcript'].str.lower()
df['summary'] = df['summary'].str.lower()

# Define a regular expression pattern to insert spaces between words and punctuation marks
pattern = r'(\w+|[^\w\s])'

# Apply the pattern to 'transcript' and 'summary' columns
df['transcript'] = df['transcript'].apply(lambda x: ' '.join(re.findall(pattern, x)))
df['summary'] = df['summary'].apply(lambda x: ' '.join(re.findall(pattern, x)))

In [7]:
df

,transcript,summary,meeting_id
0,alima . okay . um i don ' t know which one of ...,the project manager opened the meeting and int...,ES2008a
1,i used to eat it . yep . we ' ve got a cool pr...,the interface specialist and industrial design...,ES2007d
2,"did you manage ? okay . ah . hello . no , just...","for the conceptual design , the id suggested t...",TS3012c
3,"that should hopefully do the trick , um . ' ka...",the project manager recapped the decisions mad...,ES2003d
4,i ' m sorry . what was the where ' s the l_e_d...,the project manager opens the meeting by stati...,ES2015d
...,...,...,...
137,that ' s my task . okay . uh i ' ve where have...,the project manager opened the meeting by expl...,TS3010b
138,okay . hi everybody and welcome to our kick - ...,the project manager introduced the project to ...,ES2010a
139,also lots of mobile phones have got a lot of t...,the project manager opens the meeting by recap...,ES2012c
140,bonjour . why ? okay . check check check check...,the project manager reviewed the minutes from ...,TS3004c


SPlit into train and val

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming 'df' contains the DataFrame with columns 'transcript', 'summary', and 'meeting_id'

# Perform train-val split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Reset the index for both DataFrames
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [9]:
# Export DataFrame to a CSV file
df.to_csv('ami_meetings.csv', index=False)

In [10]:
train_df.to_csv('ami_meetings_train.csv', index=False)
val_df.to_csv('ami_meetings_val.csv', index=False)

Don't forget about our lonely test samples in 'ami_meetings_untouched/test/' folder...

In [11]:
# Set the path to the "test/" folder
test_folder = os.path.join("ami_meetings_untouched", "test")

# Initialize a list to store the data
data = []

# Process the test data
for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith('.transcript.txt'):
            meeting_id = file.split('.')[0]
            transcript_path = os.path.join(root, file)

            with open(transcript_path, 'r') as transcript_file:
                transcript = transcript_file.read().strip()

            data.append([transcript, meeting_id])

# Create the DataFrame
test_df = pd.DataFrame(data, columns=['transcript', 'meeting_id'])

In [12]:
test_df

,transcript,meeting_id
0,Okay. Mm-hmm. Okay. Okay. Mm-hmm. Okay. Okay. ...,IN1008
1,"If not more, yeah. Okay. Okay. So the three of...",EN2004a
2,"So if you've um if you're back to a page, then...",EN2003a
3,"Yeah, mm-hmm. Hmm. Thank you. No, I don't thin...",EN2005a
4,"Yeah. Yeah, I mean if we just want to have um ...",EN2001a
5,"Hmm? This isn't supposed to look like just Ah,...",EN2001b
6,Wonder how much of the meetings is talking abo...,EN2002a
7,Smile for the camera. Helium one. I think ther...,EN2006b
8,Yeah. Okay. Okay. Okay. Yeah. It is the initia...,IN1005
9,"I mean if you're ever, you know, late 'cause o...",EN2009c


In [14]:
import re

# Create a copy of df_test
test_df_preprocessed = test_df.copy()

# Lowercase the text in 'transcript' column
test_df_preprocessed['transcript'] = test_df_preprocessed['transcript'].str.lower()

# Define a regular expression pattern to insert spaces between words and punctuation marks
pattern = r'(\w+|[^\w\s])'

# Apply the pattern to 'transcript' column
test_df_preprocessed['transcript'] = test_df_preprocessed['transcript'].apply(lambda x: ' '.join(re.findall(pattern, x)))

In [15]:
test_df_preprocessed

,transcript,meeting_id
0,okay . mm - hmm . okay . okay . mm - hmm . oka...,IN1008
1,"if not more , yeah . okay . okay . so the thre...",EN2004a
2,"so if you ' ve um if you ' re back to a page ,...",EN2003a
3,"yeah , mm - hmm . hmm . thank you . no , i don...",EN2005a
4,"yeah . yeah , i mean if we just want to have u...",EN2001a
5,hmm ? this isn ' t supposed to look like just ...,EN2001b
6,wonder how much of the meetings is talking abo...,EN2002a
7,smile for the camera . helium one . i think th...,EN2006b
8,yeah . okay . okay . okay . yeah . it is the i...,IN1005
9,"i mean if you ' re ever , you know , late ' ca...",EN2009c


In [16]:
test_df.to_csv('ami_meetings_test_untouched.csv', index=False)
test_df_preprocessed.to_csv('ami_meetings_test.csv', index=False)